In [1]:
from keras.preprocessing import image
from keras import backend

def path_to_tensor(img_path):
    # load RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

Using TensorFlow backend.
/Users/gustavoarriaga/opt/anaconda3/envs/breedr/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/gustavoarriaga/opt/anaconda3/envs/breedr/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/gustavoarriaga/opt/anaconda3/envs/breedr/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

Using the InceptionV3 bottleneck features to classify dog breeds: https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogInceptionV3Data.npz

In [2]:
# obtain bottleneck features from another pre-trained CNN.
from keras.utils import np_utils
import numpy as np

# load bottleneck features from InceptionV3
bottleneck_features = np.load('./DogInceptionV3Data.npz')
# split data into training, validation and testing sets
train_InceptionV3 = bottleneck_features['train']
valid_InceptionV3 = bottleneck_features['valid']
test_InceptionV3 = bottleneck_features['test']

In [3]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential

# define model architecture.
doggy_model = Sequential()
doggy_model.add(GlobalAveragePooling2D(input_shape=train_InceptionV3.shape[1:]))
doggy_model.add(Dropout(.2))
doggy_model.add(Dense(64, activation='relu'))
doggy_model.add(Dropout(.1))
doggy_model.add(Dense(133, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# load the model weights with the best validation loss in prior training
doggy_model.load_weights('./weights.best.doggy_model2.hdf5')

In [5]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# set up train, test, and validation datasets
train_files, train_targets = load_dataset('./dogImages/train')
valid_files, valid_targets = load_dataset('./dogImages/valid')
test_files, test_targets = load_dataset('./dogImages/test')

# calculate classification accuracy on the test dataset
doggy_predictions = [np.argmax(doggy_model.predict(np.expand_dims(feature, axis=0))) for feature in test_InceptionV3]

# report test accuracy
test_accuracy = 100*np.sum(np.array(doggy_predictions)==np.argmax(test_targets, axis=1))/len(doggy_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 83.2536%


In [6]:
# load list of dog names
dog_names = sorted(glob("./dogImages/train/*/"))

# function that extracts weights from InceptionV3 for tensor provided
def extract_InceptionV3(tensor):
    from keras.applications.inception_v3 import InceptionV3, preprocess_input
    return InceptionV3(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

# function that takes a path to an image as input and returns the dog breed that is predicted by the model
def doggy_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_InceptionV3(path_to_tensor(img_path))
    # obtain predicted vector from extracted bottleneck features
    predicted_vector = doggy_model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

In [7]:
import cv2

face_cascade = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_alt.xml')
# function to use pre-trained face detector and return true if human faces are in image
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

In [8]:
# function that takes an image and returns the dog breed. 
def what_breed(img_path):
    # determine whether there a human in the picture
    human = face_detector(img_path)
    if human:
        return doggy_predict_breed(img_path)
    else:
        return False

In [ ]:
import random
import os

# start video capture from camera
cap = cv2.VideoCapture(0)

while(True):
    # read and rescale frames from camera
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    rgb_scaled = cv2.resize(rgb, (720, 450))
    # display images from camera in a new window
    cv2.namedWindow('Press SPACE to take a picture', cv2.WINDOW_NORMAL)
    cv2.imshow('Press SPACE to take a picture', rgb_scaled)
    # wait for a keypress
    keypress = cv2.waitKey(1)
    # if key press is SPACE process the image
    if keypress & 0xFF == ord(' '):
        # display captured image in new window
        cv2.namedWindow('This is you', cv2.WINDOW_NORMAL)
        cv2.imshow('This is you', rgb_scaled)
        # save image
        out = cv2.imwrite('capture.jpg', frame)
        # use saved image for predicting dog breed
        breed = what_breed('capture.jpg')
        # delete saved image
        os.remove("capture.jpg")
        # font for text overlay
        font = cv2.FONT_HERSHEY_SIMPLEX 
        # org for text overlay
        org = (50, 50) 
        # fontScale for text overlay
        fontScale = 1
        # red color in BGR for text overlay
        color = (0, 0, 255) 
        # line thickness of 2 px for text overlay
        thickness = 2
        if breed:
            # select a random pick of that breed
            file = random.choice(os.listdir(breed))
            #load and resize the dog pic
            dog_pic = cv2.imread(f'{breed}/{file}')
            dog_pic = cv2.resize(dog_pic, (720, 450))
            # overlay breed on dog pic
            show_breed = cv2.putText(dog_pic, breed.split('.')[2][:-1], org, font, fontScale, color, thickness, cv2.LINE_AA)
            # display dog pic and breed in a new window
            cv2.namedWindow('You look like a...', cv2.WINDOW_NORMAL)
            cv2.imshow('You look like a...', show_breed) 
        else:
            # overlay message on human pic
            show_error = cv2.putText(rgb_scaled, "No humans detected.  Let's try another picture.", org, font, fontScale, color, thickness, cv2.LINE_AA)
            # display dog pic and breed in a new window
            cv2.imshow('This is you', show_error) 
    
    # if keypress is q close windows and exit the loop
    if keypress & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
    
    